In [2]:
!pip install gast
!pip install tokenizers==0.8.1.rc1
!pip install simpletransformers
!pip install tensorflow==2.1.0

#!pip install  cudatoolkit
#!pip install  pytorch

     |████████████████████████████████| 3.0MB 4.2MB/s 
     |████████████████████████████████| 215kB 4.4MB/s 
     |████████████████████████████████| 71kB 5.7MB/s 
     |████████████████████████████████| 7.2MB 7.2MB/s 
     |████████████████████████████████| 317kB 40.2MB/s 
     |████████████████████████████████| 1.6MB 46.3MB/s 
     |████████████████████████████████| 890kB 41.0MB/s 
     |████████████████████████████████| 4.4MB 43.5MB/s 
     |████████████████████████████████| 102kB 4.4MB/s 
     |████████████████████████████████| 112kB 44.2MB/s 
     |████████████████████████████████| 163kB 37.4MB/s 
     |████████████████████████████████| 102kB 9.8MB/s 
     |████████████████████████████████| 122kB 44.8MB/s 
     |████████████████████████████████| 890kB 30.8MB/s 
     |████████████████████████████████| 1.1MB 43.6MB/s 
     |████████████████████████████████| 122kB 48.4MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36

     |████████████████████████████████| 421.8MB 40kB/s 
     |████████████████████████████████| 3.9MB 43.2MB/s 
     |████████████████████████████████| 450kB 35.0MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=56be9387ac6f2363ae2028edd33945757d7c83f85df9a7691eb8f63aed02f27a
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tens

In [3]:
import pandas as pd
import numpy as np
import itertools
from collections import Counter
import scipy
import scipy.stats as stats
import importlib, os
import sklearn
import itertools




In [4]:
eval_root = ''
repo_pairs = 'reposynergy.repositorypair.csv'.format(eval_root)
repo_pairs_scores = 'reposynergy.repositorypair_scores.csv'.format(eval_root)

repo_pairs_df = pd.read_csv(repo_pairs)
repo_pairs_df.set_index('id', inplace=True)
repo_pairs_scores_df = pd.read_csv(repo_pairs_scores)


In [6]:
data = pd.read_csv('final_repo_english_whatwhy.csv', index_col='file_id')
data.columns

Index(['local_readme_file', 'url_readme', 'created_at', 'language', 'name',
       'owner_id', 'updated_at', 'url', 'watchers', 'members_number',
       'repo_labels', 'all_languages', 'topics', 'description', 'members_ids',
       'content_text_w_o_tags', 'section_id', 'content_clean'],
      dtype='object')

In [7]:
from simpletransformers.language_representation import RepresentationModel


model = RepresentationModel(
        model_type="bert",
        model_name="bert-base-uncased",
        use_cuda=False
    )

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTextRepresentation: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a, b):
    return np.inner(a, b) / (norm(a) * norm(b))

def calculate_bert_cosine_sim(text1, text2 , model):
  # bert representation
  bert_representation = model.encode_sentences([text1, text2], combine_strategy="mean")

  # cosine similarity
  return cosine_similarity(bert_representation[0], bert_representation[1])


def _get_bert_cosine_sim(row, data, uncased):
    readme1 = data[data['url'] == row["repo_url1"]]['content_clean'].values[0]
    readme2 = data[data['url'] == row["repo_url2"]]['content_clean'].values[0]
    
    readme1 = readme1.lower() if uncased else readme1
    readme2 = readme2.lower() if uncased else readme2
    # cosine similarity between bert representations
    similarity = calculate_bert_cosine_sim(readme1, readme2, model)
    
    row['bert_uncased_sim'] =  similarity
    return row


In [18]:
#repo_pairs_scores_df = repo_pairs_scores_df.apply(_get_bert_cosine_sim , args=(data, True,), axis=1)
repo_pairs_df = repo_pairs_df.apply(_get_bert_cosine_sim, axis=1,  args=(data, True))

In [19]:
repo_pairs_scores_df.to_csv('repositorypair_score_with_bert_uncased_score.csv')
repo_pairs_df.to_csv('repositorypiar_with_bert_uncased.csv')

In [10]:
repo_pairs_scores_df[['bert_uncased_sim']].describe()

,bert_uncased_sim
count,90.000000
mean,0.761572
std,0.173057
min,0.241637
25%,0.748991
50%,0.815443
75%,0.870363
max,0.998973


In [20]:
import scipy
scipy.stats.kendalltau(repo_pairs_scores_df['score_lda_rwr_d00'], repo_pairs_scores_df['bert_uncased_sim'])
scipy.stats.spearmanr(repo_pairs_scores_df['score_lda_rwr_d00'], repo_pairs_scores_df['bert_uncased_sim'])
# there is a significantt positive correlation at p< 0,05.  SpearmanrResult(correlation=0.254353436037204, pvalue=0.01555430122607558)
# KendalltauResult(correlation=0.16181795606221216, pvalue=0.023917199458042895)

SpearmanrResult(correlation=0.254353436037204, pvalue=0.01555430122607558)